$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $

Here, we implement an algorithm "Detecting a marked vertex" from the paper Quantum walk speedup of backtracking algorithms by Ashley Montanaro. We implement operations for a complete binary tree. Subsection "Generating layers - qubit basis state labels" is demonstrating the structure of the binary tree.

Remark - from the very beginning, each gate is implemented with additional control qubit manually. For some reason automated functionality of Qiskit operator.contol() worked multiple times slower during the simulation.

<h2>Parameters</h2>

In [ ]:
num_of_layers = 5
bits_of_precision = 5

<h2>Generating layers - qubit basis state labels</h2>

In [ ]:
layers = [
    ['000'],
    ['100','001'],
    ['110','101','011','111']
]

def add_layer(layer_count,layers):
    new_layers = []
    root = '1'
    for i in range(layer_count-2):
        root = root + '0'
    root = root + '10'
    new_layers.append([root])
    for i in range(len(layers)):
        new_layers.append([])
        for j in range(2**i):
            new_layers[i+1].append('0'+layers[i][j])
        for j in range(2**i):
            new_layers[i+1].append('1'+layers[i][j])
    return new_layers

for layer_count in range(3,num_of_layers+1):
    layers = add_layer(layer_count,layers)

<h3>Verifying correctness of the tree structure</h3>

In [ ]:
correct_bit_count = True
for layer in layers:
    for node in layer:
        if(len(node) != num_of_layers + 1):
            correct_bit_count = False
print('Correct bit count in each node:',correct_bit_count)

no_duplicates = True
for layer in layers:
    for node in layer:
        count = 0
        for layer1 in layers:
            for node1 in layer1:
                if(node == node1):
                    count = count+1
        if(count != 1):
            no_duplicates = False
print('No duplicates:',no_duplicates)

correct_links = True
for i in range(num_of_layers):
    for j in range(2**i):
        vertices = [layers[i][j],layers[i+1][2*j],layers[i+1][2*j+1]]
        same_bits = []
        different_bits = []
        for k in range(num_of_layers+1):
            if(vertices[0][k] == vertices[1][k] and vertices[1][k] == vertices[2][k]):
                same_bits.append(k)
            else:
                different_bits.append(k)
        if(len(different_bits) != 2):
            correct_links = False
print('Correct links:',correct_links)

print('Root:',layers[0][0],'to',layers[1][0],'and',layers[1][1])

<h2>Array for $\psi$</h2>

In this part, we prepare states $\ket{\psi_x}$ for each vertex $x$.

In [ ]:
psi = {}

<h2>Preparing root</h2>

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    0, # 00
    sqrt(num_of_layers)/sqrt(1+2*(num_of_layers)), # 01
    1/sqrt(1+2*(num_of_layers)), # 10
    sqrt(num_of_layers)/sqrt(1+2*(num_of_layers)) #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
# you need to update only 3 operators
psi[layers[0][0]] = QuantumCircuit(num_of_layers+2)
psi[layers[0][0]].cx(0,2)
psi[layers[0][0]].cu3(2.72,0,0,0,num_of_layers) # only first parameter needs to be updated
psi[layers[0][0]].cu3(1.66,0,0,0,num_of_layers+1) # only first parameter needs to be updated
psi[layers[0][0]].ccx(0,num_of_layers+1,num_of_layers)
psi[layers[0][0]].cu3(0.421,0,0,0,num_of_layers) # only first parameter needs to be updated
psi[layers[0][0]].ccx(0,num_of_layers+1,num_of_layers)

psi[layers[0][0]].draw(output='mpl')

<h3>Generating parent-children cases</h3>

In [ ]:
cases = {}

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    0, # 00
    1/sqrt(3), # 01
    1/sqrt(3), # 10
    1/sqrt(3) #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
cases['00'] = QuantumCircuit(3)
cases['00'].cu3(3*pi/4,0,0,0,1)
cases['00'].cu3(1.91,0,0,0,2)
cases['00'].ccx(0,2,1)
cases['00'].cu3(pi/4,0,0,0,1)
cases['00'].ccx(0,2,1)

cases['00'].draw(output='mpl')

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    1/sqrt(3), # 00
    0, # 01
    1/sqrt(3), # 10
    1/sqrt(3) #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
cases['01'] = QuantumCircuit(3)
cases['01'].cu3(pi/4,0,0,0,1)
cases['01'].cu3(1.91,0,0,0,2)
cases['01'].ccx(0,2,1)
cases['01'].cu3(-pi/4,0,0,0,1)
cases['01'].ccx(0,2,1)

cases['01'].draw(output='mpl')

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    1/sqrt(3), # 00
    1/sqrt(3), # 01
    0, # 10
    1/sqrt(3) #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
cases['10'] = QuantumCircuit(3)
cases['10'].cu3(3*pi/4,0,0,0,1)
cases['10'].cu3(1.23,0,0,0,2)
cases['10'].ccx(0,2,1)
cases['10'].cu3(-pi/4,0,0,0,1)
cases['10'].ccx(0,2,1)

cases['10'].draw(output='mpl')

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    1/sqrt(3), # 00
    1/sqrt(3), # 01
    1/sqrt(3), # 10
    0 #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
cases['11'] = QuantumCircuit(3)
cases['11'].cu3(pi/4,0,0,0,1)
cases['11'].cu3(1.23,0,0,0,2)
cases['11'].ccx(0,2,1)
cases['11'].cu3(pi/4,0,0,0,1)
cases['11'].ccx(0,2,1)

cases['11'].draw(output='mpl')

<h3>Generating parent-children circuits</h3>

In [ ]:
series_of_vertex_links = []
for i in range(1,num_of_layers):
    for j in range(2**i):
        vertex_link = [layers[i][j],layers[i+1][2*j],layers[i+1][2*j+1]]
        series_of_vertex_links.append(vertex_link)

In [ ]:
for vertices in series_of_vertex_links:   
    same_bits = []
    different_bits = []
    for i in range(num_of_layers+1):
        if(vertices[0][i] == vertices[1][i] and vertices[1][i] == vertices[2][i]):
            same_bits.append(i)
        else:
            different_bits.append(i)

    current_case = ['00','01','10','11']
    for i in range(3):
        string = '' + vertices[i][different_bits[0]] + vertices[i][different_bits[1]]
        current_case.remove(string)

    psi[vertices[0]] = QuantumCircuit(num_of_layers+2)
    for i in same_bits:
        if(vertices[0][i] == '1'):
            psi[vertices[0]].cx(0,num_of_layers-i+1)
    psi[vertices[0]] = psi[vertices[0]].compose(cases[current_case[0]],[0,num_of_layers-different_bits[1]+1,num_of_layers-different_bits[0]+1])

<h3>Leaves</h3>

In [ ]:
for leaf in layers[num_of_layers]:
    psi[leaf] = QuantumCircuit(num_of_layers+2)
    reversed_index = leaf[::-1]
    for i in range(num_of_layers+1):
        if(reversed_index[i] == '1'):
            psi[leaf].cx(0,i+1)

<h2>Operators</h2>

In this part, we prepare operators $D_x$ for each vertex $x$. Here $D_x = I - 2 \ket{\psi_x} \bra{\psi_x}$

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

identity_minus_state_zero = QuantumCircuit(num_of_layers+2)

for i in range(num_of_layers+1):
    identity_minus_state_zero.cx(0,i+1)

control_states = []
for i in range(num_of_layers+1):
    control_states.append(i)


identity_minus_state_zero.h(num_of_layers+1)
identity_minus_state_zero.mct(control_states, num_of_layers+1)
identity_minus_state_zero.h(num_of_layers+1)

for i in range(num_of_layers+1):
    identity_minus_state_zero.cx(0,i+1)

d={}
for layer in layers:
    for i in layer: 
        d[i] = QuantumCircuit(num_of_layers+2)
        d[i] = d[i].compose(psi[i].inverse())
        d[i] = d[i].compose(identity_minus_state_zero)
        d[i] = d[i].compose(psi[i])

<h2>Implementation</h2>

Here we implement phase estimation algorithm. In steps where we prepare the transformation $R_BR_A$, for marked vertices $x$ we place identity operation instead of $D_x$.

We provide an array of indexes of marked elements for running the experiment. Array can be left empty for the case of no marked vertices.

In [ ]:
marked=['111111']

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi
from qiskit.circuit.library import QFT

r_a = QuantumCircuit(num_of_layers+2)

even_distance_layer = []
for i in range(0,num_of_layers+1,2):
    even_distance_layer = even_distance_layer + layers[i]

for i in even_distance_layer:
    if not i in marked:
        r_a = r_a.compose(d[i])
    
r_b = QuantumCircuit(num_of_layers+2)

odd_distance_layer = []
for i in range(1,num_of_layers+1,2):
    odd_distance_layer = odd_distance_layer + layers[i]

for i in odd_distance_layer:
    if not i in marked:
        r_b = r_b.compose(d[i])

rbra = r_a
rbra = rbra.compose(r_b)

controlled_rbra = rbra
    
phase_estimation = QuantumCircuit(num_of_layers+1+bits_of_precision,bits_of_precision)
phase_estimation.h(range(bits_of_precision))
reversed_index = layers[0][0][::-1]
for i in range(num_of_layers+1):
    if(reversed_index[i] == '1'):
        phase_estimation.x(bits_of_precision+i)

operation_qubits = []
for i in range(bits_of_precision,num_of_layers+1+bits_of_precision):
    operation_qubits = operation_qubits+[i]

qubits_to_appply = []
qubits_to_appply += [0]
qubits_to_appply += operation_qubits
for j in range(1):
    phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

for i in range(1, bits_of_precision):
    qubits_to_appply = []
    qubits_to_appply += [i]
    qubits_to_appply += operation_qubits
    controlled_rbra = controlled_rbra.compose(controlled_rbra)
    phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

phase_estimation.barrier()

phase_estimation = phase_estimation.compose(QFT(num_qubits=bits_of_precision, approximation_degree=0, do_swaps=False, inverse=True, insert_barriers=False, name='qft'),range(bits_of_precision))

phase_estimation.measure(range(bits_of_precision),range(bits_of_precision))
    
job = execute(phase_estimation,Aer.get_backend('qasm_simulator'),shots=10000)
counts = job.result().get_counts(phase_estimation)
print('marked: ['+', '.join(marked)+'], outcome: ')
n = ''
n = n.zfill(bits_of_precision)
result = counts.get(n, 0)
print(result/100,'%')
print(counts)

<h2>Running multiple experiments</h2>

We can also run multiple experiments with the code below.

In [ ]:
experiments=[
    [],
    ['111111']
]

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi
from qiskit.circuit.library import QFT

for m in range(len(experiments)):
    marked = experiments[m]

    r_a = QuantumCircuit(num_of_layers+2)

    even_distance_layer = []
    for i in range(0,num_of_layers+1,2):
        even_distance_layer = even_distance_layer + layers[i]

    for i in even_distance_layer:
        if not i in marked:
            r_a = r_a.compose(d[i])
    
    r_b = QuantumCircuit(num_of_layers+2)

    odd_distance_layer = []
    for i in range(1,num_of_layers+1,2):
        odd_distance_layer = odd_distance_layer + layers[i]

    for i in odd_distance_layer:
        if not i in marked:
            r_b = r_b.compose(d[i])

    rbra = r_a
    rbra = rbra.compose(r_b)

    controlled_rbra = rbra
    
    phase_estimation = QuantumCircuit(num_of_layers+1+bits_of_precision,bits_of_precision)
    phase_estimation.h(range(bits_of_precision))
    reversed_index = layers[0][0][::-1]
    for i in range(num_of_layers+1):
        if(reversed_index[i] == '1'):
            phase_estimation.x(bits_of_precision+i)

    operation_qubits = []
    for i in range(bits_of_precision,num_of_layers+1+bits_of_precision):
        operation_qubits = operation_qubits+[i]

    qubits_to_appply = []
    qubits_to_appply += [0]
    qubits_to_appply += operation_qubits
    for j in range(1):
        phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

    for i in range(1, bits_of_precision):
        qubits_to_appply = []
        qubits_to_appply += [i]
        qubits_to_appply += operation_qubits
        controlled_rbra = controlled_rbra.compose(controlled_rbra)
        phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

    phase_estimation.barrier()

    phase_estimation = phase_estimation.compose(QFT(num_qubits=bits_of_precision, approximation_degree=0, do_swaps=False, inverse=True, insert_barriers=False, name='qft'),range(bits_of_precision))

    phase_estimation.measure(range(bits_of_precision),range(bits_of_precision))
    
    job = execute(phase_estimation,Aer.get_backend('qasm_simulator'),shots=10000)
    counts = job.result().get_counts(phase_estimation)
    print('marked: ['+', '.join(marked)+'], outcome: ')
    n = ''
    n = n.zfill(bits_of_precision)
    result = counts.get(n, 0)
    print(result/100,'%')
    print(counts)